## Цель работы

Ознакомиться с методами одномерного поиска, используемыми в многмерных методах минимизации функции n переменных. Сравнить различные алгоритмы по эффективности на тестовых примерах.

## Задание

* Реализовать методы дихотомии, золотого сечения, исследовать их сходимость и провести сравнение по числу вычислений функций для достижения заданной точности $\epsilon$ от $10^{-1}$ до $10^{-7}$. Построить график количества вычислений минимизируемой функции от десятичного логарифма задаваемой точности $\epsilon$.
* Реализовать алгоритм поиска интервала, содержащего минимум функций.
* Реализовать метод Фибоначчи, сравнить его с методами дихотомии и золотого сечения.

In [1]:
import numpy as np
import pandas as pd
import math
from IPython.display import display

In [2]:
K1 = (3 - math.sqrt(5)) / 2
K2 = (math.sqrt(5) - 1) / 2
K3 = (1 + math.sqrt(5)) / 2

In [25]:
f = lambda x: (x - 7) ** 2

In [4]:
def FindFib(value):
    a = 1
    b = 1
    c = a + b
    while (c < value):
        v = c
        a = b
        b = v
        c = b + a
    return c, a

### Метод дихотомии

In [105]:
def DichotomyMethod(eps = 1e-7, a0 = -2.0, b0 = 20.0):
    a, b, x1, x2 = [], [], [], []
    a.append(a0)
    b.append(b0)
    x1.append((a0 + b0 - eps / 2) / 2)
    x2.append((a0 + b0 + eps / 2) / 2)
    n = 1
    while (abs(b[-1] - a[-1]) > eps):
        if (f(x1[-1]) <= f(x2[-1])):
            a.append(a[-1])
            b.append(x2[-1])
        else:
            a.append(x1[-1])
            b.append(b[-1])
        x1.append((a[n] + b[n] - eps / 2) / 2)
        x2.append((a[n] + b[n] + eps / 2) / 2)
        n += 1
    iArr = range(1, n + 1)
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n)]},
                        index = iArr)
    display(df.style.format('{:.8e}').highlight_min(subset='li', color = '#ACDDDE'))

In [106]:
DichotomyMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,8.99999997e+00,9.00000003e+00,9.99999950e-01,1.00000005e+00,-2.00000000e+00,2.00000000e+01,2.20000000e+01,1.00000000e+00
2,3.49999999e+00,3.50000004e+00,2.02500001e+01,2.02499997e+01,-2.00000000e+00,9.00000003e+00,1.10000000e+01,1.00000000e+00
3,6.24999998e+00,6.25000003e+00,3.06250007e+00,3.06249989e+00,3.49999999e+00,9.00000003e+00,5.50000004e+00,1.00000000e+00
4,7.62499998e+00,7.62500003e+00,1.40625016e-01,1.40624979e-01,6.24999998e+00,9.00000003e+00,2.75000004e+00,1.00000000e+00
5,8.31249998e+00,8.31250003e+00,9.76562354e-02,9.76562666e-02,7.62499998e+00,9.00000003e+00,1.37500005e+00,1.00000000e+00
6,7.96874998e+00,7.96875003e+00,9.76563916e-04,9.76560791e-04,7.62499998e+00,8.31250003e+00,6.87500048e-01,1.00000000e+00
7,8.14062498e+00,8.14062503e+00,1.97753841e-02,1.97753982e-02,7.96874998e+00,8.31250003e+00,3.43750049e-01,1.00000000e+00
8,8.05468748e+00,8.05468753e+00,2.99072016e-03,2.99072563e-03,7.96874998e+00,8.14062503e+00,1.71875050e-01,1.00000000e+00
9,8.01171873e+00,8.01171878e+00,1.37328568e-04,1.37329740e-04,7.96874998e+00,8.05468753e+00,8.59375498e-02,1.00000000e+00
10,7.99023435e+00,7.99023440e+00,9.53678751e-05,9.53668985e-05,7.96874998e+00,8.01171878e+00,4.29687999e-02,1.00000000e+00


### Метод золотого сечения

In [26]:
def GoldenRatioMethod(eps = 1e-7, a0 = -2.0, b0 = 20.0):

    x1, x2, fx1, fx2, a, b = [], [], [], [], [], []
    
    # step 0
    n = 0
    a.append(a0)
    b.append(b0)
    x1.append(a0 + K1 * (b0 - a0))
    x2.append(a0 + K2 * (b0 - a0))
    fx1.append(f(x1[-1]))
    fx2.append(f(x2[-1]))
    
    # step 1
    n += 1
    if fx1[0] <= fx2[0]:
        a.append(a[0])
        b.append(x2[0])
        x2.append(x1[0])
        x1.append(a[-1] + b[-1] - x1[0])
        fx1.append(f(x1[-1]))
        fx2.append(fx2[0])
    else:
        a.append(x1[0])
        b.append(b[0])
        x1.append(x2[0])
        x2.append(a[-1] + b[-1] - x2[0])
        fx1.append(fx1[0])
        fx2.append(f(x2[-1]))
    
    # step i
    while (abs(b[-1] - a[-1]) > eps):
        n += 1
        if (fx1[-1] <= fx2[-1]):
            a.append(a[-1])
            b.append(x2[-1])
            x2.append(x1[-1])
            x1.append(a[-1] + b[-1] - x1[-1])
            fx1.append(f(x1[-1]))
            fx2.append(fx2[-1])
        else:
            a.append(x1[-1])
            b.append(b[-1])
            x1.append(x2[-1])
            x2.append(a[-1] + b[-1] - x2[-1])
            fx1.append(fx1[-1])
            fx2.append(f(x2[-1]))
    iArr = range(1, n + 1)
    
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n + 1)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n + 1)]},
                        index = range(1, n + 2))
                        
    #print (len(a), len(b), len(x1), len(x2), len(fx1), len(fx2), len(range(1, n + 2)))
    display(df.style.format('{:.8e}').highlight_min(subset='li', color = '#ACDDDE'))

In [27]:
GoldenRatioMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,6.40325225e+00,1.15967478e+01,3.56107880e-01,2.11300899e+01,-2.00000000e+00,2.00000000e+01,2.20000000e+01,1.00000000e+00
2,3.19349550e+00,6.40325225e+00,1.44894765e+01,3.56107880e-01,-2.00000000e+00,1.15967478e+01,1.35967478e+01,1.00000000e+00
3,1.20975674e+00,3.19349550e+00,3.35269170e+01,1.44894765e+01,-2.00000000e+00,6.40325225e+00,8.40325225e+00,1.00000000e+00
4,3.19349550e+00,4.41951349e+00,1.44894765e+01,6.65891065e+00,1.20975674e+00,6.40325225e+00,5.19349550e+00,1.00000000e+00
5,4.41951349e+00,5.17723427e+00,6.65891065e+00,3.32247492e+00,3.19349550e+00,6.40325225e+00,3.20975674e+00,1.00000000e+00
6,5.17723427e+00,5.64553147e+00,3.32247492e+00,1.83458501e+00,4.41951349e+00,6.40325225e+00,1.98373876e+00,1.00000000e+00
7,5.64553147e+00,5.93495505e+00,1.83458501e+00,1.13432075e+00,5.17723427e+00,6.40325225e+00,1.22601798e+00,1.00000000e+00
8,5.93495505e+00,6.11382866e+00,1.13432075e+00,7.85299639e-01,5.64553147e+00,6.40325225e+00,7.57720782e-01,1.00000000e+00
9,6.11382866e+00,6.22437863e+00,7.85299639e-01,6.01588502e-01,5.93495505e+00,6.40325225e+00,4.68297198e-01,1.00000000e+00
10,6.22437863e+00,6.29270228e+00,6.01588502e-01,5.00270071e-01,6.11382866e+00,6.40325225e+00,2.89423585e-01,1.00000000e+00


### Метод Фибоначчи

In [81]:
def FibonachiMethod(eps = 10 ** (-7), a0 = -2.0, b0 = 20.0):
    fn2, fn0 = FindFib((b0 - a0) / eps)
    a, b, x1, x2, xm = [], [], [], [], []
    a.append(a0)
    b.append(b0)
    x1.append(a[0] + (b[0] - a[0]) * fn0 / fn2)
    x2.append(a[0] + b[0] - x1[0])
    xm.append(0)
    n = 0
    while (abs(b[n] - a[n]) > eps):
        if (f(x1[n]) <= f(x2[n])):
            a.append(a[n])
            b.append(x2[n])
            x2.append(x1[n])
            x1.append(a[n + 1] + b[n + 1] - x1[n])
            xm.append(x1[n])
        else:
            a.append(x1[n])
            b.append(b[n])
            x1.append(x2[n])
            x2.append(a[n + 1] + b[n + 1] - x2[n])
            xm.append(x2[n])
        n += 1
    iArr = range(1, n + 2)
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n + 1)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n + 1)]},
                        index = iArr)
    display(df.style.format('{:.8e}').highlight_min(subset='li', color = '#ACDDDE'))

In [82]:
FibonachiMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,6.40325225e+00,1.15967478e+01,2.54960339e+00,1.29365944e+01,-2.00000000e+00,2.00000000e+01,2.20000000e+01,1.00000000e+00
2,3.19349550e+00,6.40325225e+00,2.31024855e+01,2.54960339e+00,-2.00000000e+00,1.15967478e+01,1.35967478e+01,1.00000000e+00
3,6.40325225e+00,8.38699101e+00,2.54960339e+00,1.49762042e-01,3.19349550e+00,1.15967478e+01,8.40325225e+00,1.00000000e+00
4,8.38699101e+00,9.61300899e+00,1.49762042e-01,2.60179800e+00,6.40325225e+00,1.15967478e+01,5.19349550e+00,1.00000000e+00
5,7.62927023e+00,8.38699101e+00,1.37440564e-01,1.49762042e-01,6.40325225e+00,9.61300899e+00,3.20975674e+00,1.00000000e+00
6,7.16097303e+00,7.62927023e+00,7.03966256e-01,1.37440564e-01,6.40325225e+00,8.38699101e+00,1.98373876e+00,1.00000000e+00
7,7.62927023e+00,7.91869381e+00,1.37440564e-01,6.61069614e-03,7.16097303e+00,8.38699101e+00,1.22601798e+00,1.00000000e+00
8,7.91869381e+00,8.09756743e+00,6.61069614e-03,9.51940243e-03,7.62927023e+00,8.38699101e+00,7.57720782e-01,1.00000000e+00
9,7.80814384e+00,7.91869381e+00,3.68087861e-02,6.61069614e-03,7.62927023e+00,8.09756743e+00,4.68297198e-01,1.00000000e+00
10,7.91869381e+00,7.98701745e+00,6.61069614e-03,1.68546532e-04,7.80814384e+00,8.09756743e+00,2.89423585e-01,1.00000000e+00


### Поиск интервала, содержащего минимум функции

In [45]:
def FindMinimum(a0 = -2.0, b0 = 20.0):
    k = 0
    x = []
    x.append(a0)
    h = 0
    delt = 10 ** (-8)
    if (f(x[0]) > f(x[0] + delt)):
        k += 1
        x.append(x[0] + delt)
        h = delt
    elif (f(x[0]) < f(x[0] + delt)):
        k += 1
        x.append(x[0] - delt)
        h = -delt
    else:
        print ("Govno kakoe-to")
        exit
    
    flag = True
    while (flag):
        h *= 2
        x.append(x[k] + h)
        if (f(x[k]) > f(x[k + 1])):
            k += 1
        else:
            flag = False
            print ('[', x[k - 2], ';', x[k], ']')

In [46]:
FindMinimum()

[ 0.6843545499999999 ; 8.73741823 ]
